In [1]:
pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.4 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
from sklearn.metrics import confusion_matrix,classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
neg_seq = []
neg_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Colab Notebooks/TRneg250.txt", "fasta"):
    neg_seq.append(str(seq_record.seq))
    neg_id.append(str(seq_record.id))
pos_seq = []
pos_id= []

for seq_record in SeqIO.parse("/content/drive/MyDrive/Colab Notebooks/TRpos250.txt", "fasta"):
    pos_seq.append(str(seq_record.seq))
    pos_id.append(str(seq_record.id))
seqs=pos_seq + neg_seq
dfclass = pd.DataFrame({'class' :  np.repeat((1,0), (250, 250))})
y=dfclass['class']
print(seqs)

['MANTPVNYQLTRTANAIPEIFVGGTFAEIKQNLIEWLNGQNEFLDYDFEGSRLNVLCDLLAYNTLYIQQFGNAAVYESFMRTANLRSSVVQAAQDNGYLPTSKSAAQTEIMLTCTDALNRNYITIPRGTRFLAYAKDTSVNPYNFVSREDVIAIRDKNNQYFPRLKLAQGRIVRTEIIYDKLTPIIIYDKNIDRNQVKLYVDGAEWINWTRKSMVHAGSTSTIYYMRETIDGNTEFYFGEGEISVNASEGALTANYIGGLKPTQNSTIVIEYISTNGADANGAVGFSYADTLTNITVININENPNDDPDFVGADGGGDPEDIERIRELGTIKRETQQRCVTATDYDTFVSERFGSIIQAVQTFTDSTKPGYAFIAAKPKSGLYLTTVQREDIKNYLKDYNLAPITPSIISPNYLFIKTNLKVTYALNKLQESEQWLEGQIIDKIDRYYTEDVEIFNSSFAKSKMLTYVDDADHSVIGSSATIQMVREVQNFYKTPEAGIKYNNQIKDRSMESNTFSFNSGRKVVNPDTGLEEDVLYDVRIVSTDRDSKGIGKVIIGPFASGDVTENENIQPYTGNDFNKLANSDGRDKYYVIGEINYPADVIYWNIAKINLTSEKFEVQTIELYSDPTDDVIFTRDGSLIVFENDLRPQYLTIDLEPISQ', 'MPETPIMGQDVKYLFQSIDAATGSAPLFPAYQTDGSVSGERELFDEQTKNGRILGPGSVADSGEVTYYGKRGDAGQKAIEDAYQNGKQIKFWRVDTVKNENDKYDAQFGFAYIESREYSDGVEGAVEISISLQVIGELKNGEIDTLPEEIVNVSKGGYDFQQPGQTTGEAPGTVPLPNAPQNLTYTATTDSVTVKWDAVEGADSYNVYRGAEKKLDANVTTTSHTLTGIQPDTQLTVNVAAVNAGGESPMSQIVTKTLPAESTG', 'MTVSTEVDHNDYTGNGVTTSFPYTFRIFKKSDLVVQVVDLNENITELILDTDYTVTGAGGYTCGDV

In [5]:
Indneg_seq = []
Indneg_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Colab Notebooks/TSneg63.txt", "fasta"):
    Indneg_seq.append(str(Indseq_record.seq))
    Indneg_id.append(str(Indseq_record.id))
Indpos_seq = []
Indpos_id= []

for Indseq_record in SeqIO.parse("/content/drive/MyDrive/Colab Notebooks/TSpos63.txt", "fasta"):
    Indpos_seq.append(str(Indseq_record.seq))
    Indpos_id.append(str(Indseq_record.id))

Indseqs=Indpos_seq + Indneg_seq
Inddfclass = pd.DataFrame({'class' :  np.repeat((1,0), (63, 63))})
z=Inddfclass['class']
print(Indseqs)

['MEKVKAWLIKYKWWIVAAIGGLAAFLLLKNRGGGSGGGGEYMVGSGPVYQQAGSGAVDNTMALAALQANTQLSAQNAQLQAQMDASRLQLETQLNIETLAADNAHYSTQSQLQLGMAQVDLSKYLGDLQSTTSTALAGMQSDTAKYQSNIQLQAENIRANTSLAEIDAQKYIVGKQADIAKYQAKTERRGQDYGFALGLLNFGGKFF', 'MKLDYNSREIFFGNEALIVADMSKGINGKPEFTNHKIVAGLVSVGSMEDQAETNSYPADDVPDHGVKKGATLLQGEMVFIQTDQALKEDILGQQRTENGLGWSPTGNWKTKCVQYLIKGRKRDKVTGEFVDGYRVVVYPNLTPTAEATKESETDSVDGVDPIQWTLAVQATESDIYLNGGKKVPAIEYEIWGEQAKDFVKKMESGLFIMQPDTVLAGAITLVAPVIPNVTTATKGNNDGTIVVPDTLKDSKGGTVKVTSVIKDAHGKVATNGQLAPGVYIVTFSADGYEDVTAGVSVTDHS', 'MSIFSSLFKVIKKVISKVVATLKKIFKKIWPLLLIVAIIYFAPYLAGFFTSAGFTGIGGIFSSIATTITPTLTSFLSTAWSGVGSLASTAWSGFQSLGMGTQLAVVSGAAALIAPEETAQLVTEIGTTVGDIAGTIIGGVAKALPGWIWIAAGGLAVWALWPSSDSKE', 'MKAMKQRIAKFSPVASFRNLCIAGSVTAATSLPAFAGVIDTSAVESAITDGQGDMKAIGGYIVGALVILAVAGLIYSMLRKA', 'MFNLKVKDLNGSARGLTQAFAIGELKNQLSVGALQLPLQFTRTFSASMTSELLWEVGKGNIDPVMYARLFFQYAQAGGALSVDELVNQFTEYHQSTACNPEIWRKLTAYITGSSNRAIKADAVGKVPPTAILEQLRTLAPSEHELFHHITTDFVCHVLSPLGFILPDAAYVYRVGRTATYPNFYALVDCVRASDLRRMLTALSSVDSKMLQATFKAKGALAPAL

In [6]:
pip install keras-tuner --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.1 MB/s eta 0:00:00


In [7]:
import keras_tuner as kt

In [8]:
max_words = 5000
max_len = 350
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(seqs)
sequences = tok.texts_to_sequences(seqs)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [9]:
#Independent Testing
indtok = Tokenizer(num_words=max_words)
indtok.fit_on_texts(Indseqs)
Indsequences = indtok.texts_to_sequences(Indseqs)
Indsequences_matrix = sequence.pad_sequences(Indsequences,maxlen=max_len)
X_train=sequences_matrix
y_train=y
X_test=Indsequences_matrix
y_test=z

In [10]:
import keras_tuner as kt
from tensorflow import keras

In [11]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(hp.Choice('units', [2, 4, 8, 16, 32, 64, 128]),activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [12]:
tuner1 = kt.RandomSearch(
    build_model,
    overwrite=True,
    objective='val_accuracy',
    max_trials=10,
    directory='./multiclass_classifier/training')

In [13]:
tuner1.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
best_model = tuner1.get_best_models()[0]

Trial 7 Complete [00h 00m 03s]
val_accuracy: 0.5

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 39s


In [14]:
tuner1.results_summary()

Results summary
Results in ./multiclass_classifier/training/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
units: 8
Score: 1.0

Trial 04 summary
Hyperparameters:
units: 4
Score: 1.0

Trial 05 summary
Hyperparameters:
units: 128
Score: 0.9920634627342224

Trial 03 summary
Hyperparameters:
units: 64
Score: 0.976190447807312

Trial 00 summary
Hyperparameters:
units: 16
Score: 0.9682539701461792

Trial 02 summary
Hyperparameters:
units: 32
Score: 0.9444444179534912

Trial 06 summary
Hyperparameters:
units: 2
Score: 0.5


In [15]:
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (500, 8)                  2808      
                                                                 
 dense_1 (Dense)             (500, 1)                  9         
                                                                 
Total params: 2817 (11.00 KB)
Trainable params: 2817 (11.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
accr = best_model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

4/4 [==============================] - 0s 4ms/step - loss: 0.6188 - accuracy: 1.0000
Test set
  Loss: 0.619
  Accuracy: 1.000


In [17]:
pred=best_model.predict(X_test)
pred1 = np.round_(pred)

from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)

4/4 [==============================] - 0s 3ms/step
MCC:  1.0


array([[63,  0],
       [ 0, 63]])

In [18]:
from sklearn.metrics import matthews_corrcoef as mcc
mc=mcc(y_test, pred1)
print("MCC: ",mc)
from sklearn.metrics import confusion_matrix as cm
cm(y_test, pred1)
print(classification_report(y_test, pred1))
tp, fn, fp, tn = confusion_matrix(y_test, pred1).ravel()
print("MCC ---> {0}".format(mc))
print("Confusion Matrix. tn, fp, fn, tp ---> ", tp, fn, fp, tn)
print("Precision --->TP/TP+Fp ", tp/(tp+fp))
print("Recall - SN - Sensitivity --->TP/TP+FN ", tp/(tp+fn))
print("Specificity - SP ---> ", tn/(tn+fp))
print("Balanced Accuracy ---> ", ((tp/(tp+fn))+(tn/(tn+fp)))/2)
print("Jaccard Index --->TP/TP+FN+FP ", tp/(tp+fn+fp))

MCC:  1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        63
           1       1.00      1.00      1.00        63

    accuracy                           1.00       126
   macro avg       1.00      1.00      1.00       126
weighted avg       1.00      1.00      1.00       126

MCC ---> 1.0
Confusion Matrix. tn, fp, fn, tp --->  63 0 0 63
Precision --->TP/TP+Fp  1.0
Recall - SN - Sensitivity --->TP/TP+FN  1.0
Specificity - SP --->  1.0
Balanced Accuracy --->  1.0
Jaccard Index --->TP/TP+FN+FP  1.0
